In [1]:
from Datos import Datos
from ModeloPINN import ModeloPINN

In [2]:
modelo = ModeloPINN('..\..\data\sistema_solar.csv',['mercurio','venus','tierra','marte','jupiter','saturno','urano','neptuno'],fecha = 'fecha', device='cpu')

In [3]:
modelo.posiciones_velocidades(obj='tierra')
modelo.escalar_y_transformar()
modelo.particionar_datos(tam_test=0.2, batch_size=64)

In [4]:
modelo.construir_red(neuronas=[1+8*3,128,128,128,64,32,16,3])
modelo.optimizar_red(paciencia = 35, min_lr=1e-6)

In [ ]:
modelo.entrenar_cv(k_folds = 5, epochs = 1000, peso_fis = 0, early = 100)

In [ ]:
peso = modelo.calcular_peso_fisico()
historiales = modelo.entrenar_cv(k_folds = 5, epochs = 500, peso_fis = peso, early = 100)